In [119]:
include("../src/StMrfTracking.jl")
import StMrfTracking

using ProgressMeter
using PyCall
using PyPlot
import Images

@pyimport imageio

T = StMrfTracking.Tracking;
SM = StMrfTracking.StMrf;
L = StMrfTracking.Labeler;
B = StMrfTracking.ImgBlock;

In [88]:
const video_file = "/home/viktor/VirtualBox VMs/Shared/VehicleTracking/vehicle_videos/4K_p2.mp4"
const json_file = "/home/viktor/.julia/v0.6/StMrfTracking/scripts/config_4k_down.json"
max_out_frames = -1
config = JSON.parsefile(json_file);
const suppress_shadows = true
const quite = true

const slit_x = config["slit"]["x"];
const slit_y = config["slit"]["y"];
const slit_width = config["slit"]["width"];
const vehicle_direction = config["slit"]["vehicle_direction"];

const frame_step = config["video"]["frame_step"];
const n_background_init_frames = config["video"]["n_background_init_frames"];
const n_background_init_iters = config["video"]["n_background_init_iters"];

const block_width = config["blocks"]["width"];
const block_height = config["blocks"]["height"];

const threshold = config["algorithm"]["background_diff_threshold"];

In [89]:
reader = imageio.get_reader(video_file);
if max_out_frames <= 0
    max_out_frames = reader[:get_meta_data]()["nframes"]
end

if !quite
    T.plot_frame(T.preprocess_frame(reader[:get_data](0)), slit_x, slit_y, slit_width, block_height)
    PyPlot.show()
end

In [5]:
const frames = frames = T.read_all_data(reader; frame_step=frame_step, max_frames=max_out_frames);
background = T.init_background(frames[1:min(n_background_init_frames, size(frames, 1))]; max_iters=n_background_init_iters);

Reading data...100%|████████████████████████████████████| Time: 0:00:58
Background initialization...100%|███████████████████████| Time: 0:01:22


In [120]:
blocks = B.init_field(size(background)[1:2], (block_height, block_width))

slit_coords = [B.get_block_coords(slit_y, slit_x + i * block_width, block_height, block_width) for i in 1:floor(Int, slit_width / block_width)]
slit_line = [blocks[c...] for c in slit_coords];

const out_suffix = suppress_shadows ? ".out_sh" :  ".out";
out_video_file = join(splitext(video_file), out_suffix);
writer = imageio.get_writer(out_video_file, fps=reader[:get_meta_data]()["fps"] / frame_step);

println("Start video processing...")
new_object_id = 1;

Start video processing...


In [121]:
f_id = 1
@showprogress 1 "Processing video..." for f_id in 1:(size(frames, 1) - 1)
    global frame = frames[f_id];
    global prev_pixel_map = B.blocks_to_object_map(blocks);
    SM.update_slit_objects!(blocks, slit_coords, frame, background, new_object_id; threshold=threshold);
    
    global old_frame = frame;
    global frame = frames[f_id + 1];

    global obj_ids_map = map(b -> b.object_id, blocks);

    global object_ids = sort(unique(obj_ids_map));
    global object_ids = object_ids[object_ids .> 0]
    global group_coords = [collect(zip(findn(obj_ids_map .== id)...)) for id in object_ids]
    global motion_vecs = [SM.find_motion_vector(blocks, frame, old_frame, gc) for gc in group_coords]
    
    global labels = zeros(Int, size(blocks))
    if size(motion_vecs, 1) != 0
        global motion_vecs_rounded = [SM.round_motion_vector(mv, block_width, block_height) for mv in motion_vecs]
        global foreground = T.subtract_background(frame, background, threshold)

        if suppress_shadows
            global foreground = T.suppress_shadows(foreground, frame, background)
        end
        global new_map = SM.update_object_ids(blocks, obj_ids_map, motion_vecs_rounded, group_coords, foreground);

        if vehicle_direction == "up"
            new_map[map(x -> x.end_y, blocks) .> slit_y + block_height] = Set();
        else
            new_map[map(x -> x.end_y, blocks) .< slit_y] = Set();
        end

        labels = L.label_map_gco(blocks, new_map, motion_vecs, prev_pixel_map, frame, old_frame);
        labels = Images.label_components(labels, trues(3, 3))
    end
    
    new_object_id = maximum(labels) + 1
    B.set_block_ids!(blocks, labels)
    
    obj_map = B.blocks_to_object_map(blocks)
    out_frame = T.plot_frame_to_array(frame, slit_x, slit_y, slit_width, block_height; object_map=obj_map, plot_boxes=true)
    writer[:append_data](out_frame)
end

[swscaler @ 0x130c1e0] Warning: data is not aligned! This can lead to a speedloss
Processing video...100%|████████████████████████████████| Time: 0:03:57


In [126]:
writer[:close]()